In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris,load_wine
from DecisionTree import D_Tree 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from stratifion import stratified_sampling
from MetaCost import Metacost
from ucimlrepo import fetch_ucirepo
from KNN_plus import KNN

In [2]:
def make_cost_matrix(label):
    # np.random.seed(0)
    label_list,counts = np.unique(label,return_counts=True)
    class_number = len(label_list)
    data = np.zeros((class_number,class_number))
    Data = pd.DataFrame(data,columns=label_list,index=label_list)
    for name1 in label_list:
        for name2 in label_list:
            if name1 == name2:
                Data[name1][name2] = np.random.randint(0,1000)
            else:
                Data[name1][name2] = np.random.randint(0,2000*counts[np.where(label_list==name1)] / counts[np.where(label_list==name2)])
                # Data[name1][name2] = np.random.uniform(0,10000)
    return Data

In [3]:
def compute_cs(cost_matrix,true_label,predict_label):
    cs = 0

    for i in range(len(true_label)):
        
        # if true_label[i]!=predict_label[i]:
        cs += cost_matrix.iloc[predict_label[i]][true_label[i]]

    # conf_matrix = confusion_matrix(true_label, predict_label)
    # total_cost = np.sum(conf_matrix * cost_matrix.values)

    # return total_cost/len(true_label)
    return cs/len(true_label)
    

In [14]:
  
# fetch dataset 
statlog_australian_credit_approval = fetch_ucirepo(id=143) 
  
# data (as pandas dataframes) 
X = statlog_australian_credit_approval.data.features.values
y = statlog_australian_credit_approval.data.targets.values 

credit_ = pd.DataFrame(X)
credit_['Y'] = y
cs = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    credit_train_set,credit_test_set = train_test_split(credit_,test_size=0.3,random_state=42)
    credit_test_data = credit_test_set.iloc[:,:-1].values
    credit_test_label = credit_test_set.iloc[:,-1].values
    tree = D_Tree(credit_train_set)
    tree.fit()
    predict_label = tree.predict(credit_test_data)
    cs.append(compute_cs(cost_matrix,credit_test_label,predict_label))
cs = np.array(cs)
# print(cs)
print("C4.5",np.mean(cs))
print("C4.5",np.std(cs))

cs_ = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    credit_train_set,credit_test_set = train_test_split(credit_,test_size=0.3,random_state=42)
    credit_test_data = credit_test_set.iloc[:,:-1].values
    credit_test_label = credit_test_set.iloc[:,-1].values
    train_set_x,train_set_y = stratified_sampling(credit_train_set,cost_matrix.values,'undersampling') 
    train_set = pd.DataFrame(train_set_x)
    train_set['Y'] = train_set_y
    tree = D_Tree(train_set)
    tree.fit()
    predict_label = tree.predict(credit_test_data)
    cs_.append(compute_cs(cost_matrix,credit_test_label,predict_label))
cs_ = np.array(cs_)
# print(cs_)
print("under",np.mean(cs_))
print("under",np.std(cs_))

cs_ = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    credit_train_set,credit_test_set = train_test_split(credit_,test_size=0.3,random_state=42)
    credit_test_data = credit_test_set.iloc[:,:-1].values
    credit_test_label = credit_test_set.iloc[:,-1].values
    train_set_x,train_set_y = stratified_sampling(credit_train_set,cost_matrix.values,'oversampling') 
    train_set = pd.DataFrame(train_set_x)
    train_set['Y'] = train_set_y
    tree = D_Tree(train_set)
    tree.fit()
    predict_label = tree.predict(credit_test_data)
    cs_.append(compute_cs(cost_matrix,credit_test_label,predict_label))
cs_ = np.array(cs_)
# print(cs_)
print("over",np.mean(cs_))
print("over",np.std(cs_))

cs___=[]
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    credit_train_set,credit_test_set = train_test_split(credit_,test_size=0.3,random_state=42)
    credit_test_data = credit_test_set.iloc[:,:-1].values
    credit_test_label = credit_test_set.iloc[:,-1].values
    tree = DecisionTreeClassifier(criterion='entropy')
    meta = Metacost(credit_train_set,tree,cost_matrix.values,q=False)
    model = meta.fit('Y')
    predict_label = model.predict(credit_test_data)
    cs___.append(compute_cs(cost_matrix,credit_test_label,predict_label))
cs___ = np.array(cs___)
# print(cs__)
print("meta",np.mean(cs___))
print("meta",np.std(cs___))

cs_k = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    credit_train_set,credit_test_set = train_test_split(credit_,test_size=0.3,random_state=42)
    credit_test_data = credit_test_set.iloc[:,:-1].values
    credit_test_label = credit_test_set.iloc[:,-1].values
    knn = KNN(2,credit_train_set,50)
    knn.fit(cost_matrix.values,n_neighbor=20,euclidean_features=[1,2,6,9,12,13],hamming_features=[0,3,4,5,7,8,10,11])
    predict_label = knn.predict(credit_test_set)
    cs_k.append(compute_cs(cost_matrix,credit_test_label,predict_label))
cs_k = np.array(cs_k)
print("my_algorithm",np.mean(cs_k))
print("my_algorithm",np.std(cs_k))

C4.5 629.630193236715
C4.5 194.7012777618022
under 541.6828502415459
under 174.08003920242433
over 601.1043478260871
over 137.93675126157
meta 423.60410628019326
meta 199.2296681528701
my_algorithm 457.2999999999999
my_algorithm 200.40274158956083


In [15]:
pima = pd.read_csv('pima.dat',header=None)
x = pima.iloc[:,:-1].values
y = pima.iloc[:,-1].values
unique = np.unique(y)
change = {unique[i]:i for i in range(len(unique))}
y = [change[i] for i in y]
pima_ = pd.DataFrame(x)
pima_['Y'] = y

cs = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    pima_train_set,pima_test_set = train_test_split(pima_,test_size=0.3)
    pima_test_data = pima_test_set.iloc[:,:-1].values
    pima_test_label = pima_test_set.iloc[:,-1].values
    tree = D_Tree(pima_train_set)
    tree.fit()
    predict_label = tree.predict(pima_test_data)
    
    cs.append(compute_cs(cost_matrix,pima_test_label,predict_label))
cs = np.array(cs)
print("C4.5",cs.mean())
print("C4.5",cs.std())

cs_ = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    pima_train_set,sonar_test_set = train_test_split(pima_,test_size=0.3)
    pima_test_data = pima_test_set.iloc[:,:-1].values
    pima_test_label = pima_test_set.iloc[:,-1].values
    train_set_x,train_set_y = stratified_sampling(pima_train_set,cost_matrix.values,'undersampling') 
    train_set = pd.DataFrame(train_set_x)
    train_set['Y'] = train_set_y
    tree = D_Tree(train_set)
    tree.fit()
    predict_label = tree.predict(pima_test_data)
    cs_.append(compute_cs(cost_matrix,pima_test_label,predict_label))
cs_ = np.array(cs_)
print("under",cs_.mean())
print("under",cs_.std())

cs_ = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    pima_train_set,sonar_test_set = train_test_split(pima_,test_size=0.3)
    pima_test_data = pima_test_set.iloc[:,:-1].values
    pima_test_label = pima_test_set.iloc[:,-1].values
    train_set_x,train_set_y = stratified_sampling(pima_train_set,cost_matrix.values,'oversampling') 
    train_set = pd.DataFrame(train_set_x)
    train_set['Y'] = train_set_y
    tree = D_Tree(train_set)
    tree.fit()
    predict_label = tree.predict(pima_test_data)
    cs_.append(compute_cs(cost_matrix,pima_test_label,predict_label))
cs_ = np.array(cs_)
print("over",cs_.mean())
print("over",cs_.std())

# cs___=[]
# for i in range(20):
#     cost_matrix = make_cost_matrix(y)
#     pima_train_set,sonar_test_set = train_test_split(pima_,test_size=0.3,random_state=42)
#     pima_test_data = pima_test_set.iloc[:,:-1].values
#     pima_test_label = pima_test_set.iloc[:,-1].values
#     tree = DecisionTreeClassifier(criterion='entropy')
#     meta = Metacost(pima_train_set,tree,cost_matrix.values,q=False)
#     model = meta.fit('Y')
#     predict_label = model.predict(pima_test_data)
#     cs___.append(compute_cs(cost_matrix,pima_test_label,predict_label))
# cs___ = np.array(cs___)
# # print(cs__)
# print("meta",np.mean(cs___))
# print("meta",np.std(cs___))

cs_k = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    pima_train_set,pima_test_set = train_test_split(pima_,test_size=0.3,random_state=42)
    pima_test_data = pima_test_set.iloc[:,:-1].values
    pima_test_label = pima_test_set.iloc[:,-1].values
    knn = KNN(6,sonar_train_set,50)
    print([i for i in range(pima_train_set.shape[1]-1)])
    knn.fit(cost_matrix.values,n_neighbor=20,euclidean_features=[i for i in range(pima_train_set.shape[1]-1)])
    predict_label = knn.predict(pima_test_set)
    cs_k.append(compute_cs(cost_matrix,pima_test_label,predict_label))
cs_k = np.array(cs_k)
print("my_algorithm",np.mean(cs_k))
print("my_algorithm",np.std(cs_k))

C4.5 672.5811688311687
C4.5 273.2904477628713
under 540.4619047619047
under 232.44715590680497
over 575.8090909090909
over 257.57611643868927


NameError: name 'sonar_train_set' is not defined